# Import Packages

In [1]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=888e18f07dba1da13e704d4f99fa9a2d0a7bedc3df2a8abb47616b3244b444ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-ca4c0tpn/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
import pandas as pd
import json
import spacy
import random
from spacy.tokens import Doc
from spacy.util import minibatch, compounding
import en_core_web_lg
import en_core_web_sm
import warnings

# Mount Drive

In [4]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/'


Mounted at /content/drive


# Import Data

In [9]:
with open (abspath_curr + 'annotations_training.json') as f:
    training_data = json.load(f)

with open (abspath_curr + 'annotations_testing.json') as f:
    testing_data = json.load(f)

In [10]:
len(training_data['annotations'])

1603

In [11]:
len(testing_data['annotations'])

540

In [12]:
training_data.pop('classes')


['FOOD']

In [27]:
testing_data['annotations']

[["'mix dry salad dressing mix , oil , and lemon pepper. put pretzels in a huge container. pour oil mixture over pretzels and toss very well. let sit and toss again every half hour or so for two hours. bag up and hide !'\r",
  {'entities': [[5, 27, 'FOOD'],
    [30, 33, 'FOOD'],
    [40, 53, 'FOOD'],
    [58, 66, 'FOOD'],
    [93, 104, 'FOOD'],
    [110, 118, 'FOOD']]}],
 ["'pound chicken breasts on each side and place in a ziplock baggie. add lemonade and lemon pepper. marinate overnight. drain chicken and grill. season with salt& more lemon pepper if desired'\r",
  {'entities': [[7, 22, 'FOOD'],
    [71, 79, 'FOOD'],
    [84, 97, 'FOOD'],
    [124, 131, 'FOOD'],
    [155, 159, 'FOOD'],
    [166, 178, 'FOOD']]}],
 ['\'toast the waffles as directed on the package\', "joey likes hers barely warm , so they\'re still pretty flexible", \'spread peanut butter and jam on one waffle. put the other waffle on top. let it sit for a minute so the peanut butter gets melty and "drooly"\'\r',
  {'en

In [14]:
# TRAIN_DATA = []
# for text, annotations in training_data['annotations']:
#     for annotation in annotations['entities']: 
#         print(annotation)

In [15]:
# TRAIN_DATA

# Train Model

In [16]:
# def test_model(model, text):
#     doc = nlp(text)
#     results = []
#     entities = []
#     for ent in doc.ents:
#         entities.append((ent.start_char, ent.end_char, ent.label_))
#     if len(entities) > 0:
#         results = [text, {"entities": entities}]
#         return (results)

In [17]:
# for text, annotations in training_data['annotations']:
#     print(annotations)

In [18]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print (losses)
    return (nlp)

In [19]:
# add NER to the pipeline and the new label
nlp = en_core_web_lg.load()
ner = nlp.get_pipe("ner")
ner.add_label("FOOD")

# get the names of the components we want to disable during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# start the training loop, only training NER
epochs = 50
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    sizes = compounding(1.0, 4.0, 1.001)
    
    # batch up the examples using spaCy's minibatc
    for epoch in range(epochs):
        examples = training_data['annotations']
        batches = minibatch(examples, size=sizes)
        losses = {}
        
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)

        print("Losses ({}/{})".format(epoch + 1, epochs), losses)

Losses (1/50) {'ner': 53312.77113045441}
Losses (2/50) {'ner': 49973.55497547565}
Losses (3/50) {'ner': 49934.85618459713}
Losses (4/50) {'ner': 49406.68548493646}
Losses (5/50) {'ner': 49361.45523967047}
Losses (6/50) {'ner': 49071.84367079218}
Losses (7/50) {'ner': 48891.31361849804}
Losses (8/50) {'ner': 49019.67990632914}
Losses (9/50) {'ner': 48720.933436752064}
Losses (10/50) {'ner': 48595.231560506625}
Losses (11/50) {'ner': 48544.59153161815}
Losses (12/50) {'ner': 48933.4695452936}
Losses (13/50) {'ner': 48438.762067593634}
Losses (14/50) {'ner': 48451.32641175669}
Losses (15/50) {'ner': 48409.05915786}
Losses (16/50) {'ner': 48323.50880426477}
Losses (17/50) {'ner': 48257.5824244041}
Losses (18/50) {'ner': 48367.79663031828}
Losses (19/50) {'ner': 48247.919836244546}
Losses (20/50) {'ner': 48466.99775585308}
Losses (21/50) {'ner': 48438.170667960076}
Losses (22/50) {'ner': 48373.403631520574}
Losses (23/50) {'ner': 48136.59215144257}
Losses (24/50) {'ner': 47889.24799969647}


In [20]:
nlp.to_disk("ner_model")

In [21]:
!zip -r /content/model.zip /content/ner_model

  adding: content/ner_model/ (stored 0%)
  adding: content/ner_model/parser/ (stored 0%)
  adding: content/ner_model/parser/moves (deflated 56%)
  adding: content/ner_model/parser/cfg (deflated 49%)
  adding: content/ner_model/parser/model (deflated 7%)
  adding: content/ner_model/meta.json (deflated 46%)
  adding: content/ner_model/tokenizer (deflated 79%)
  adding: content/ner_model/tagger/ (stored 0%)
  adding: content/ner_model/tagger/cfg (deflated 34%)
  adding: content/ner_model/tagger/tag_map (deflated 50%)
  adding: content/ner_model/tagger/model (deflated 7%)
  adding: content/ner_model/vocab/ (stored 0%)
  adding: content/ner_model/vocab/vectors (deflated 8%)
  adding: content/ner_model/vocab/strings.json (deflated 61%)
  adding: content/ner_model/vocab/lookups.bin (deflated 45%)
  adding: content/ner_model/vocab/lexemes.bin (deflated 64%)
  adding: content/ner_model/vocab/key2row (deflated 13%)
  adding: content/ner_model/ner/ (stored 0%)
  adding: content/ner_model/ner/move

In [22]:
doc = nlp('I like steaks and lemons.')
for ent in doc.ents:
  print(ent.text, ent.label_)

steaks FOOD
lemons. FOOD


# Evaluate Model

In [28]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run
example = testing_data['annotations']

ner_model = spacy.load("ner_model") # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, example)
print(results)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'intj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'amod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'conj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'prep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'pobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'xcomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'npadvmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nummod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'prt': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'mark': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ccomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'acomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'quantmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'aux': {'p': 0.0, 'r':